In [46]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten,Conv2D,Activation,Dropout
from keras import backend as K
import keras
from keras.models import Sequential, Model
from keras.models import load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import MaxPool2D


In [47]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory='/home/jszumny/Documents/copy of smoke data/hpwren/modeldata/training/',target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory='/home/jszumny/Documents/copy of smoke data/hpwren/modeldata/testing/', target_size=(224,224))

Found 22273 images belonging to 3 classes.
Found 5598 images belonging to 3 classes.


In [48]:
def VGG16():
    model = Sequential()
    model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
    model.add(Flatten(name='flatten'))
    model.add(Dense(256, activation='relu', name='fc1'))
    model.add(Dense(128, activation='relu', name='fc2'))
    model.add(Dense(1, activation='sigmoid', name='output'))
    return model

model = VGG16()
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_52 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_53 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_54 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_55 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 56, 56, 128)      0         
 g2D)                                                 

In [49]:
Vgg16 = Model(inputs=model.input, outputs=model.get_layer('vgg16').output)

In [51]:
Vgg16.load_weights("/home/jszumny/Documents/copy of smoke data/hpwren/modeldata/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [52]:
for layer in Vgg16.layers:
    layer.trainable = False

In [53]:
for layer in model.layers:
    print(layer, layer.trainable)


<keras.layers.convolutional.conv2d.Conv2D object at 0x7f41274db6d0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f4111b9d120> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f415c32b040> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f4111b9d3f0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f41259fdd20> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f413fd77730> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f41259fd840> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f41259fcb80> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f4125979b10> False
<keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f41259d7cd0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f41259a04f0> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f4111bd1720> False
<keras.layers.convolutional.conv2d.Conv2D object at 0x7f4111bd2650> False
<keras.layers.poo

In [54]:
opt = SGD(learning_rate=1e-4, momentum=0.9)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
es=EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
mc = ModelCheckpoint('/home/jszumny/Documents/copy of smoke data/hpwren/modeldata/best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='max', save_freq='epoch')

In [56]:
H = model.fit_generator(steps_per_epoch=50,generator=traindata, validation_data= testdata, validation_steps=10,epochs=100,callbacks=[mc,es])

/tmp/ipykernel_919672/1953829058.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(steps_per_epoch=50,generator=traindata, validation_data= testdata, validation_steps=10,epochs=100,callbacks=[mc,es])
2023-06-05 15:43:06.218683: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/100
50/50 [==============================] - ETA: 0s - loss: 0.8525 - accuracy: 0.5967

2023-06-05 15:46:30.655778: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_accuracy improved from -inf to 0.61667, saving model to /home/jszumny/Documents/copy of smoke data/hpwren/modeldata/best_model.h5
50/50 [==============================] - 247s 5s/step - loss: 0.8525 - accuracy: 0.5967 - val_loss: 0.8262 - val_accuracy: 0.6167
Epoch 2/100
50/50 [==============================] - ETA: 0s - loss: 0.7649 - accuracy: 0.6113
Epoch 2: val_accuracy did not improve from 0.61667
50/50 [==============================] - 247s 5s/step - loss: 0.7649 - accuracy: 0.6113 - val_loss: 0.7756 - val_accuracy: 0.6073
Epoch 3/100
50/50 [==============================] - ETA: 0s - loss: 0.7279 - accuracy: 0.6198
Epoch 3: val_accuracy did not improve from 0.61667
50/50 [==============================] - 245s 5s/step - loss: 0.7279 - accuracy: 0.6198 - val_loss: 0.7327 - val_accuracy: 0.6156
Epoch 4/100
50/50 [==============================] - ETA: 0s - loss: 0.7182 - accuracy: 0.6246
Epoch 4: val_accuracy improved from 0.61667 to 0.62604, saving model to /home/j

KeyboardInterrupt: 